In [1]:
import os
import sys
import time
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf
slim = tf.contrib.slim
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import osvos
from dataset import Dataset


In [2]:
seq_name = "bmx-trees"
file_name = seq_name+"singledotr10-20ite"
test_frames = sorted(os.listdir(os.path.join('DAVIS', 'JPEGImages', '480p', seq_name)))
test_imgs = [os.path.join('DAVIS', 'JPEGImages', '480p', seq_name, frame) for frame in test_frames]

gpu_id = 0
train_model = True 

result_path = os.path.join('DAVIS', 'Results', 'Segmentations', '480p', 'OSVOS', file_name)

In [ ]:
# Train parameters
parent_path = os.path.join('models', 'OSVOS_parent', 'OSVOS_parent.ckpt-50000')
#logs_path = os.path.join('models', seq_name)

logs_path = os.path.join('models', file_name)
max_training_iters = 4

osvos.train_init(parent_path, logs_path, max_training_iters,
                                 save_step, display_step, global_step, iter_mean_grad=1, ckpt_name=seq_name, resume_training=True)

if train_model:
    train_imgs = [os.path.join('DAVIS', 'JPEGImages', '480p', seq_name, '00000.jpg')+' '+
                  #os.path.join('DAVIS', 'Annotations', '480p', seq_name, '00000.png')]
                  #os.path.join('DAVIS', 'Results', 'Segmentations', '480p', 'OSVOS', seq_name+'20', '00000.png')]
                  os.path.join('DAVIS', 'Annotations', 'scribble', file_name, '00000.png')]
    dataset = Dataset(train_imgs, test_imgs, './', data_aug=True)
else:
    dataset = Dataset(None, test_imgs, './')

# Train the network
if train_model:
    # More training parameters
    learning_rate = 1e-8
    save_step = max_training_iters
    side_supervision = 3
    display_step = 2
    duration = 0
    with tf.Graph().as_default():
        with tf.device('/gpu:' + str(gpu_id)):
            global_step = tf.Variable(0, name='global_step', trainable=False)
            start = time.time()
            osvos.train_finetune(dataset, parent_path, side_supervision, learning_rate, logs_path, max_training_iters,
                                 save_step, display_step, global_step, iter_mean_grad=1, ckpt_name=seq_name, resume_training=True)
            duration = time.time()-start 
            print "duration: %.3f" % duration